# Animating the Execution of Nondeterministic Finite State Machines

Import `gvanim`, which was installed using `pip install GraphvizAnim`. This module is used to generate the interactive animations.

In [8]:
from gvanim import Animation
from gvanim.jupyter import interactive
ga = Animation() # variable used to represent the animation 

In the section below, we introduce all the classes and methods needed to produce a Nondeterministic Finite State Machine. These methods were referenced from the lecture notes (02 Regular Languages). 

In [2]:
class FiniteStateMachine:
    def __init__(self, T, Q, R, q0, F):
        self.T, self.Q, self.R, self.q0, self.F = T, Q, R, q0, F
    def __repr__(self):
        return str(self.q0) + '\n' + ' '.join(self.F) + '\n' + \
               '\n'.join(r[0] + ' ' + r[1] + ' → ' + r[2] for r in self.R)

def parseFSM(fsm: str) -> FiniteStateMachine:
    fsm = [line for line in fsm.split('\n') if line.strip() != '']
    q0 = fsm[0].split()[0] # first line: initialstate
    F = set(fsm[1].split()) # second line: finalstate, finalstate, ...
    R = set()
    for line in fsm[2:]: # all subsequent lines: "source symbol → target"
        l, r = line.split('→')
        R |= {(l.split()[0], l.split()[1], r.split()[0])}
    T = {r[1] for r in R}
    Q = {q0} | F | {r[0] for r in R} | {r[2] for r in R}
    return FiniteStateMachine(T, Q, R, q0, F)

class Choice:
    def __init__(self, e1, e2): self.e1, self.e2 = e1, e2
class Conc:
    def __init__(self, e1, e2): self.e1, self.e2 = e1, e2
class Star:
    def __init__(self, e): self.e = e
        
def syntaxgraph(re):
    global node, T
    if re == '': return {(None, None)}
    elif type(re) == str:
        node += 1; T.add(re); return {(None, (re, str(node))), ((re, str(node)), None)}
    elif type(re) == Choice:
        return syntaxgraph(re.e1) | syntaxgraph(re.e2)
    elif type(re) == Conc:
        g1, g2 = syntaxgraph(re.e1), syntaxgraph(re.e2)
        return {(a, b) for (a, b) in g1 if b} | \
               {(a, b) for (a, b) in g2 if a} | \
               {(a, b) for (a, c) in g1 for (d, b) in g2 if not c and not d}
    elif type(re) == Star:
        g = syntaxgraph(re.e)
        return {(None, None)} | g | \
               {(a, b) for (a, c) in g for (d, b) in g if not c and not d}
    else: raise Exception('not a regular expression')
        
def convertRegExToFSM(re):
    global node, T; node, T = 0, set()
    g = syntaxgraph(re)
    Q = {str(n) for n in range(node + 1)}
    R = {('0', b[0], b[1]) for (a, b) in g if not a and b} | \
        {(a[1], b[0], b[1]) for (a, b) in g if a and b}
    F = {a[1] for (a, b) in g if a and not b} | ({'0'} if (None, None) in g else set())
    output = FiniteStateMachine(T, Q, R, '0', F)
    output = str(output)
    return output

###### Defining the Nondeterministic FSM 

First, we need to define the NFA using the methods defined above. We do this by defining a regular expression, and convert that regex into an FSM using `convertRegExToFSM()`. We also split the FSM into a list of strings. 

In [3]:
E3 = Choice(Conc('a', 'b'), Conc('a', 'c')); A3 = convertRegExToFSM(E3); A3 = A3.splitlines()
print(A3)
for i in A3:
    print(i)
# start state = 0 
# final states = 2, 4
# transitions = a, b, c 
# all states = 0, 1, 2, 3, 4 

['0', '2 4', '0 a → 1', '3 c → 4', '1 b → 2', '0 a → 3']
0
2 4
0 a → 1
3 c → 4
1 b → 2
0 a → 3


### Printing the Model:

In this section, we define a method for printing a model of the NFA.<br> 
The method `printNFSM(A)` takes a list of strings as input and produces a model of the NFA, constructed from a series of edges and nodes.  

In [4]:
def printNFSM(A):
    event = [] # order of events
    if type(A) == list: # only proceeds if input is correct type  
        for i in range(2,len(A)):
            if type(A[i]) == str: # only proceeds if input is list of strings 
                old = A[i][0:1] 
                transition = A[i][2:3] 
                new = A[i][A[i].find("→")+2:]
                ga.add_edge(old,new) # extend edge from old to new 
                ga.label_edge(old,new,transition) # add label for transition 
                event.append([old,transition,new]) 
            else: # returns error message if incorrect type  
                raise Exception('error: list of strings expected')
        return (event) 
    else: # returns error message if input type is incorrect 
        raise Exception('error: list expected')
        
printNFSM(A3)

[['0', 'a', '1'], ['3', 'c', '4'], ['1', 'b', '2'], ['0', 'a', '3']]

### Animating the Model:

In this stage, we define a method for animating the model of the NFA.<br> 
The method `animateNFSM(A)` takes a list of strings as input, and calls the previous defined method `printNFSM(A)` using this input. It iterates through the list of lists outputted from `printNFSM(A)`, highlighting each old state, transition, and new state in the NFA. 

In [5]:
def animateNFSM(A):
    if type(A) == list: # only proceeds if input is correct type 
        start = list(A[0].strip("{}")) # list of starting states
        final = A[1].replace('{','').replace('}','').split(' ') # list of final states
        event = printNFSM(A) # calls printNFSM(), and uses the output it generates 
        next = []
        event.sort(key = lambda event: event[2]) 
        for i in event: 
            if type(i) == list: # only proceeds if input is list of lists 
                ga.highlight_node(i[0]) # highlights old state 
                ga.next_step()
                ga.highlight_edge(i[0],i[2]) # highlights edge 
                ga.next_step()
                ga.highlight_node(i[2]) # highlights new state 
            else: # returns error message if incorrect type
                raise Exception('error: list expected')
        return event
    else: # returns error message if input type is incorrect 
        raise Exception('error: list expected')    

animateNFSM(A3)

[['0', 'a', '1'], ['1', 'b', '2'], ['0', 'a', '3'], ['3', 'c', '4']]

### Running the Animation:

Here, we call the method `interactive()`, which was imported with the module `gvanim`. This method generates the interative animation using the previously defined methods, `printNFSM()` and `animateNFSM()`.

Move the slider from left to right to view the animation of the NFA. 

In [9]:
interactive( ga, 600 )

interactive(children=(IntSlider(value=0, description='n', max=0), Output()), _dom_classes=('widget-interact',)…